In [1]:
import pandas as pd
import numpy as np
import time, os, sys, gc, json

from Models.framework_utils import *
from Models.diagnostic_utils import *

# load numerapi
# public_id, secret_key = get_numerapi_config()
# napi, modelnameids = get_napi_and_models(public_id, secret_key)

# load data
ds_version = "v4.2"
dataset_loc = os.path.join(os.getcwd(), 'train_datasets', ds_version)
# currentRound = get_update_training_data(napi, dataset_loc, ds_version)

np.random.seed(42)
print("# Loading data... ",end='')

# validation data V*
validation, VI, targets = processData(os.path.join(dataset_loc, 'validation_int8.parquet'), return_target_names=True)

with open(os.path.join(dataset_loc, "features.json"), "r") as f:
    feature_metadata = json.load(f)
feature_sets = feature_metadata['feature_sets']

BVP = pd.read_parquet(os.path.join(dataset_loc, 'validation_benchmark_models.parquet'),engine="fastparquet")
ids = BVP.index.values

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
# Loading data... 

In [2]:
import Models
Mods = [Models.__dict__[x] for x in Models.models]
print(Mods)

[<module 'Models.DCNMultiMix' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNMultiMix.py'>, <module 'Models.DCNNbase' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNNbase.py'>, <module 'Models.DCNNtmix' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNNtmix.py'>, <module 'Models.DCNNtmnt' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNNtmnt.py'>, <module 'Models.DCNV4' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNV4.py'>, <module 'Models.DCNV4EXP' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNV4EXP.py'>, <module 'Models.DCNV5e14' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNV5e14.py'>, <module 'Models.DCNV5e17' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNV5e17.py'>, <module 'Models.DCNV5e9' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DCNV5e9.py'>, <module 'Models.DNNV6' from 's:\\DS Projects\\Competitions\\Numerai-v2\\Models\\DNNV6.py'>, <module 'Models.ENA' from '

In [3]:
prediction_names = []
for mod in Mods:
    for submit_name in mod.submit_on:
        prediction_names.append(mod.name + "_" + submit_name)

AttributeError: module 'Models.DCNMultiMix' has no attribute 'submit_on'

In [ ]:
prediction_names

['LightGBM_yourmodelnamehere']

In [ ]:
predictions = get_validation_predictions()

In [ ]:
def updateModelValidationPredictions(model):
    # get the corresponding predictions for all submit_on
    VP = np.full((len(validation), len(model.submit_on)), np.nan)

    for i, submit_name in enumerate(model.submit_on):
        full_name = model.name + "_" + submit_name
        if full_name in predictions:
            preds = predictions[full_name]
            VP[:len(preds), i] = preds

    # find the latest era where we don't have predictions
    startEra = 0
    for I in VI:
        if np.isnan(VP[I]).any(): break
        startEra += 1

    print("making predictions for "+model.name+"...",end=' ')
    # make predictions
    if model.ensembled:
        newPreds = model.predict(validation, VI[startEra:], feature_sets, {key: value[startEra:] for key, value in predictions.items()})
    else:
        newPreds = model.predict(validation, VI[startEra:], feature_sets)
    if len(newPreds.shape) == 1: newPreds = newPreds.reshape(-1, 1)
    VP[startEra:, :] = newPreds

    for i, submit_name in enumerate(model.submit_on):
        predictions[model.name + "_" + submit_name] = VP
    print("done")

    # save predictions
    saveValidationPredictions(VP, model, ids)

for model in Mods:
    if not model.ensembled:
        updateModelValidationPredictions(model)
for model in Mods:
    if model.ensembled:
        updateModelValidationPredictions(model)



    
# produce pretty graphs and diagnostics for everything
# should find better name for Models directory/module

making predictions... done
saving validation predictions for LightGBM ['yourmodelnamehere']
                 id  prediction
0  n000101811a8a843    0.490641
1  n001e1318d5072ac    0.502775
2  n002a9c5ab785cbb    0.491644
# Writing predictions to validation_LightGBM_yourmodelnamehere.csv... done


In [ ]:
predictions

{'LightGBM_yourmodelnamehere': array([[0.49064106],
        [0.5027753 ],
        [0.49164446],
        ...,
        [0.5030672 ],
        [0.5284548 ],
        [0.49681808]])}